In [9]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
import nest_asyncio
nest_asyncio.apply()

from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse, StreamingResponse
from PIL import Image
import json
import asyncio
import io
import os
from datetime import datetime
import time

app = FastAPI()

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/process-multiple-frames-stream")
async def simple_frame_reader_with_save(request: Request):
    """Simple endpoint that reads frames AND saves them to frames folder, then streams text"""
    
    try:
        print("=" * 50)
        print("🔄 READING AND SAVING FRAMES...")
        
        # Get content type
        content_type = request.headers.get("content-type", "")
        print(f"📋 Content-Type: {content_type}")
        
        # Parse form data
        form_data = await request.form()
        print(f"📥 Form data items: {len(form_data)}")
        print(f"🔍 Form keys: {list(form_data.keys())}")
        
        # Create frames directory
        os.makedirs("frames", exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Process and save each frame
        saved_frames = []
        frame_count = 0
        
        for key, value in form_data.items():
            if key.startswith('frame_') and hasattr(value, 'read'):
                try:
                    # Read file data
                    file_data = await value.read()
                    print(f"📝 Processing {key}: {len(file_data)} bytes")
                    
                    # Open image
                    img = Image.open(io.BytesIO(file_data))
                    
                    # Save frame with timestamp
                    save_path = f"frames/frame_{timestamp}_{key}.png"
                    img.save(save_path)
                    
                    saved_frames.append({
                        "key": key,
                        "filename": getattr(value, 'filename', 'unknown'),
                        "size_bytes": len(file_data),
                        "image_size": f"{img.width}x{img.height}",
                        "saved_path": save_path
                    })
                    
                    frame_count += 1
                    print(f"✅ Saved {key}: {save_path} ({img.width}x{img.height})")
                    
                except Exception as frame_error:
                    print(f"❌ Error processing {key}: {frame_error}")
            
            elif key == 'frame_count':
                expected_count = str(value)
                print(f"📊 Expected frame count: {expected_count}")
        
        print(f"🎉 Successfully saved {frame_count} frames to frames/ folder!")
        
        # Now create streaming response with initial success data + streaming text
        async def generate_stream():
            # First yield the success response
            initial_response = {
                "success": True,
                "message": f"Successfully received and saved {frame_count} frames!",
                "frames_saved": saved_frames,
                "timestamp": timestamp,
                "save_directory": "frames/",
                "streaming": True,
                "frame_count": frame_count,
                "type": "initial"
            }
            yield f"data: {json.dumps(initial_response)}\n\n"
            
            # Small delay before starting analysis
            await asyncio.sleep(0.3)
            
            # Then stream mock GPT-like Two Sum solution
            mock_gpt_responses = [
                "🤖 I can see you're working on the Two Sum problem! Let me help you solve this step by step.\n",
                "📋 **Problem Analysis:**\nGiven an array of integers `nums` and an integer `target`, return indices of two numbers that add up to `target`.\n",
                "💡 **Approach 1: Brute Force**\nWe could use nested loops to check every pair, but that would be O(n²) time complexity.\n",
                "⚡ **Better Approach: Hash Map**\nLet's use a hash map to solve this in O(n) time!\n",
                "```python\ndef twoSum(nums, target):\n    num_map = {}\n    for i, num in enumerate(nums):\n        complement = target - num\n        if complement in num_map:\n            return [num_map[complement], i]\n        num_map[num] = i\n```\n",
                "🔍 **How it works:**\n1. Create an empty hash map\n2. For each number, calculate its complement (target - current number)\n3. Check if complement exists in hash map\n4. If yes, return the indices; if no, store current number and index\n",
                "📊 **Time Complexity:** O(n) - single pass through array\n**Space Complexity:** O(n) - hash map storage\n",
                "🧪 **Test with your example:**\n`nums = [2,7,11,15], target = 9`\n- i=0, num=2, complement=7, not in map, store {2:0}\n- i=1, num=7, complement=2, found at index 0, return [0,1]\n",
                "✨ **Alternative One-liner (Python):**\n```python\ndef twoSum(nums, target):\n    seen = {}\n    return next(([seen[target-n], i] for i, n in enumerate(nums) if target-n in seen or seen.setdefault(n, i)), None)\n```\n",
                "🎯 **Key Insights:**\n- Hash maps provide O(1) average lookup time\n- We only need one pass through the array\n- Always check if complement exists before storing current number\n",
                "🏆 **Solution Complete!** This approach efficiently solves Two Sum in linear time.\n"
            ]
            
            for i, response in enumerate(mock_gpt_responses):
                await asyncio.sleep(0.8)  # Slightly longer delay for reading
                stream_data = {
                    "type": "stream",
                    "content": response,
                    "step": i + 1,
                    "total_steps": len(mock_gpt_responses)
                }
                yield f"data: {json.dumps(stream_data)}\n\n"
            
            # Final completion message with detected text
            final_data = {
                "type": "complete",
                "content": "🎯 Two Sum solution explained successfully!\n",
                "total_frames_processed": frame_count,
                "detected_text": f"""**Detected from {frame_count} frames:**

**LeetCode Problem 1: Two Sum**

**Problem Statement:**
Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.

You may assume that each input would have exactly one solution, and you may not use the same element twice.

**Example 1:**
Input: nums = [2,7,11,15], target = 9
Output: [0,1]
Explanation: Because nums[0] + nums[1] == 9, we return [0, 1].

**Example 2:**
Input: nums = [3,2,4], target = 6
Output: [1,2]

**Constraints:**
- 2 ≤ nums.length ≤ 10⁴
- -10⁹ ≤ nums[i] ≤ 10⁹
- -10⁹ ≤ target ≤ 10⁹
- Only one valid answer exists.

**Follow-up:** Can you come up with an algorithm that is less than O(n²) time complexity?

**Code Editor shows:**
```python
class Solution:
    def twoSum(self, nums: List[int], target: int) -> List[int]:
        # Your solution here
        pass
```"""
            }
            yield f"data: {json.dumps(final_data)}\n\n"
            
            # Send the [DONE] signal that frontend is waiting for
            yield "data: [DONE]\n\n"
        
        return StreamingResponse(
            generate_stream(),
            media_type="text/event-stream",
            headers={
                "Cache-Control": "no-cache",
                "Connection": "keep-alive",
                "Content-Type": "text/event-stream",
                "Access-Control-Allow-Origin": "*",
                "Access-Control-Allow-Methods": "*",
                "Access-Control-Allow-Headers": "*"
            }
        )
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return JSONResponse({
            "success": False,
            "error": f"Processing failed: {str(e)}"
        })

# Start server
async def start_frame_saver_server():
    import uvicorn
    try:
        print("🚀 Starting FRAME READER & SAVER server on http://localhost:8000")
        print("📁 Frames will be saved to: frames/ directory")
        print("📡 Now includes streaming Two Sum solution!")
        
        print("💡 Send your frontend request to save frames and get streaming output!")
        
        config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
        server = uvicorn.Server(config)
        await server.serve()
    except Exception as e:
        print(f"❌ Server error: {e}")

await start_frame_saver_server()

🚀 Starting FRAME READER & SAVER server on http://localhost:8000
📁 Frames will be saved to: frames/ directory
📡 Now includes streaming Two Sum solution!
💡 Send your frontend request to save frames and get streaming output!


INFO:     Started server process [2240]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🔄 READING AND SAVING FRAMES...
📋 Content-Type: multipart/form-data; boundary=----WebKitFormBoundaryivQwvqdCgPCdjqFa
🔄 READING AND SAVING FRAMES...
📋 Content-Type: multipart/form-data; boundary=----WebKitFormBoundaryOwGe72zdBarJqWfB
📥 Form data items: 5
🔍 Form keys: ['frame_0', 'frame_1', 'frame_2', 'frame_3', 'frame_count']
📝 Processing frame_0: 1153982 bytes
✅ Saved frame_0: frames/frame_20250708_145034_frame_0.png (1280x720)
📝 Processing frame_1: 1152638 bytes
✅ Saved frame_1: frames/frame_20250708_145034_frame_1.png (1280x720)
📥 Form data items: 5
🔍 Form keys: ['frame_0', 'frame_1', 'frame_2', 'frame_3', 'frame_count']
📝 Processing frame_2: 1159615 bytes
✅ Saved frame_2: frames/frame_20250708_145034_frame_2.png (1280x720)
📝 Processing frame_0: 1153982 bytes
✅ Saved frame_0: frames/frame_20250708_145035_frame_0.png (1280x720)
📝 Processing frame_1: 1152638 bytes
✅ Saved frame_1: frames/frame_20250708_145035_frame_1.png (1280x720)
📝 Processing frame_3: 1195354 bytes
✅ Saved frame_3: fr

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [2240]
